In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

from textblob import TextBlob
from googletrans import Translator

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pyvi import ViTokenizer, ViUtils


In [2]:
# Đọc file CSV
file_path = "sentiments_v2.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print(data.columns)

Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')


In [3]:
# Đọc file CSV
file_path = "sentiments_v2.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Chỉ kiểm tra nếu cột 'comment' tồn tại
if 'comment' in data.columns:
    # Khởi tạo công cụ dịch
    translator = Translator()

    # Hàm dịch và phân tích cảm xúc
    def analyze_sentiment(text):
        # Dịch văn bản sang tiếng Anh
        translated_text = translator.translate(text, src='vi', dest='en').text
        # Phân tích cảm xúc bằng TextBlob
        blob = TextBlob(translated_text)
        return blob.sentiment.polarity  # Trả về điểm cảm xúc từ -1 (tiêu cực) đến 1 (tích cực)

    # Áp dụng hàm cho 5 bình luận đầu tiên
    data['sentiment'] = data['comment'].head(5).apply(analyze_sentiment)

    # Hiển thị 5 bình luận đầu tiên với điểm cảm xúc
    print(data[['comment', 'sentiment']].head(5))
else:
    print("Cột 'comment' không tồn tại trong DataFrame.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')
                                             comment  sentiment
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   0.295833
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   0.391667
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  -0.025000
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   0.850000
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  -0.125000


In [4]:
import time

# Đọc file CSV
file_path = "sentiments_v2.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Chỉ kiểm tra nếu cột 'comment' tồn tại
if 'comment' in data.columns:
    # Khởi tạo công cụ dịch
    translator = Translator()

    # Hàm dịch và phân tích cảm xúc
    def analyze_sentiment(text):
        # Kiểm tra nếu văn bản không phải là None hoặc chuỗi rỗng
        if isinstance(text, str) and text.strip():
            try:
                # Dịch văn bản sang tiếng Anh
                translated_text = translator.translate(text, src='vi', dest='en').text
                # Phân tích cảm xúc bằng TextBlob
                blob = TextBlob(translated_text)
                return blob.sentiment.polarity  # Trả về điểm cảm xúc từ -1 (tiêu cực) đến 1 (tích cực)
            except Exception as e:
                print(f"Lỗi khi dịch văn bản: {e}")
                return None  # Trả về None nếu có lỗi
        else:
            return None  # Trả về None cho các văn bản không hợp lệ

    # Tạo cột mới 'sentiment' và áp dụng hàm cho toàn bộ cột 'comment'
    data['sentiment'] = data['comment'].apply(lambda x: analyze_sentiment(x) if x is not None else None)

    # Hiển thị 5 bình luận đầu tiên với điểm cảm xúc
    print(data[['comment', 'sentiment']].head(5))

    # Lưu DataFrame trở lại file CSV với cột mới
    data.to_csv("sentiments_v2_with_sentiment.csv", index=False)
else:
    print("Cột 'comment' không tồn tại trong DataFrame.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')
                                             comment  sentiment
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   0.295833
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   0.391667
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  -0.025000
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   0.850000
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  -0.125000
